## 数据处理

In [1]:
import pandas as pd
import io
import re
from datetime import datetime, timedelta
from typing import List, Union, Optional
from datetime import datetime

import os, re, json, time
import typing as T
import requests

from pathlib import Path
from openpyxl import Workbook, load_workbook

from openpyxl.utils import get_column_letter
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, Border, Side, PatternFill
from data_processing import load_and_process,build_jsonl_for_range, save_jsonl


In [2]:
## 研发字典
speaker_map = {
    "16186514":   "peter本尊",
    "1655611808": "运营绾绾",
    "2073820674": "沙利文老师",
    "2726067525": "milissa",
}
## 客服字典
MAPPING_FILE = "mapping地球1.xlsx"

##QQ的txt文件
pathtxt   = "1231《欢迎来到地球》测试2群.txt"

# 设定时间范围
start_time = "2025-12-01 00:00:00"
end_time   = "2025-12-31 00:00:00"


# 1) 拿到 JSONL（列表）
jsonl_lines01 = build_jsonl_for_range(
    pathtxt=pathtxt,
    mapping_file=MAPPING_FILE,
    speaker_map=speaker_map,
    start_time=start_time,
    end_time=end_time,
    return_str=False,   # 返回 list[str]
)

print(len(jsonl_lines01), "lines")
print(jsonl_lines01[0])



47104 lines
{"发言日期": "2025-12-01", "发言时间": "00:00:30", "客服ID": "青瓷游戏客服-哼唧(3008694133)", "客服消息": "哼"}


## 大模型分类

## 定义

In [3]:
"""
批处理 10000 条聊天数据（每批 100 条）：
- 模型#1：过滤非游戏相关（只保留相关 JSON 行，原样输出）
- 模型#2：提取高讨论的发言并分析
- 结果按word格式文档输出
"""
from model_classifyV1_Copy1_Copy1 import (
    load_system_prompt,
    build_user_prompt_filter,build_user_prompt_clsuter,call_ark_chat_completions,
    extract_valid_json_lines,add_index_to_jsonl_lines,count_output_filter_stats,get_covered_indices_from_cluster_output,
    aggregate_cluster_outputs,build_user_prompt_cluster_agg,assign_global_cluster_ids,
    extract_top5_heat_clusters,attach_discussion_points,extract_cluster_stats,append_daily_top5_to_version_jsonl,infer_date_for_batch,
    match_dialogs_by_time,build_user_prompt_subcluster_opinion,extract_time_axis_from_title,
    parse_and_normalize_opinion_output,build_daily_top5_opinion_records,build_user_prompt_version_agg,parse_model2_output_to_json_list,normalize_model3_clusters,
    read_jsonl_file,build_version_agg_input_jsonl_text,compute_version_heat_topk,clusters_list_to_jsonl,
    compute_all_clusters_point_metrics,extract_version_top5_clusters_from_point_metrics,print_mech_time_from_version_top5,get_dialogs_lines_by_dt_list_debug,
    build_user_prompt_version_opinion,parse_opinion_output_to_list,merge_version_top5_with_opinions,build_cluster_heat_summary,
    build_heat_trend_input_jsonl,build_user_prompt_heat_trend,merge_version_final_summary)

## 设置参数

In [4]:
# ============= 你的模型与文件配置（改这里） =============
API_URL   = "https://ark.cn-beijing.volces.com/api/v3/chat/completions" 
API_KEY = "de91deb0-aae6-46cb-bac0-17ac3b6107f5" #API
V3_MODEL_ID= "ep-20251020160142-5d7hp"#接入点
V3_1_MODEL_ID = "ep-20251020160025-9p5tj"#接入点
R1_MODEL_ID = "ep-20251020160103-5n6g2"#接入点


# ✅ 统一：V2 作为唯一工程根目录
BASE_DIR = Path(r"E:\项目\玩家社群分析智能体\玩家发言整理（供运营侧）\版本总结(多天V2)").resolve()

PROMPT_MD_PATH01 = BASE_DIR / "提示词1.md"
PROMPT_MD_PATH02 = BASE_DIR / "2话题分类.md"
PROMPT_MD_PATH03 = BASE_DIR / "3日聚合.md"
PROMPT_MD_PATH04 = BASE_DIR / "版本聚合.md"
PROMPT_MD_PATH05 = BASE_DIR / "5讨论点聚合.md"
PROMPT_MD_PATH06 = BASE_DIR / "6版本讨论点分析.md"
PROMPT_MD_PATH07 = BASE_DIR / "7热度趋势分析.md"

VERSION_TOP5_JSONL = (BASE_DIR / "test_top5.jsonl").resolve()

BATCH_SIZE       = 300
SLEEP_BETWEEN    = 1   # 每批之间的间隔，防止QPS触发限流；按需调整
RETRIES          = 2
TEMPERATURE      = 0.20
MAX_TOKENS       = 16384
TIMEOUT_SEC      = 600
# =====================================================

## 1.检验每日话提簇分类输出

In [15]:
import json
import time
from tqdm import tqdm



# ===========================================================
# =============== 主循环：模型#1 + 模型#2 ====================
# ===========================================================

# 读取系统提示
system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 做话题簇
# 1⃣ 给原始 jsonl 每条加 _idx
jsonl_lines01_indexed = add_index_to_jsonl_lines(jsonl_lines01)

total = len(jsonl_lines01_indexed)

PRINT_UNCLUSTERED = False
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

# 统计总量用的（可选）
total_filtered_lines = 0         # 模型#1 输出总行数之和（含客服）
total_filtered_player_lines = 0  # 模型#1 输出玩家行数之和
total_covered_idx = 0            # 被话题簇覆盖的原始发言总数（按 _idx 去重）


for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    # 本批原始发言（已带 _idx）
    batch_lines = jsonl_lines01_indexed[start:end]

    # 本批原始 _idx 集合，用来和话题簇覆盖做差集
    batch_original_idx = set()
    for line in batch_lines:
        try:
            obj = json.loads(line)
            batch_original_idx.add(int(obj["_idx"]))
        except Exception:
            continue

    try:
        # ========== 模型 #1：筛相关 ==========
        user_prompt1 = build_user_prompt_filter(batch_lines)
        output_filter = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt01,
            user_prompt=user_prompt1,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        if not output_filter:
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue

        # 统计筛相关之后的行数：总行数 & 玩家行数
        filter_total_lines, filter_player_lines = count_output_filter_stats(output_filter)
        total_filtered_lines += filter_total_lines
        total_filtered_player_lines += filter_player_lines

        tqdm.write(
            f"[批次 {b+1}] 模型#1 输出总行数：{filter_total_lines}，"
            f"其中玩家发言：{filter_player_lines} 条（不去重）"
        )

        # ========== 模型 #2：话题簇 ==========
        user_prompt2 = build_user_prompt_clsuter(output_filter)
        output_cluster = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt02,
            user_prompt=user_prompt2,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        if not output_cluster:
            tqdm.write(f"[批次 {b+1}] 模型#2 无有效输出，跳过。")
            continue

        # 打印本批的话题簇输出（原样）
        print(f"\n===== 批次 {b+1}/{total_batches} 话题簇输出 =====")
        print(output_cluster)
        print("===========================================\n")

        # 计算：本批有哪些 _idx 被话题簇覆盖（从“发言行号列表”解析）
        covered_idx = get_covered_indices_from_cluster_output(output_cluster)
        total_covered_idx += len(covered_idx)

        # 和“本批原始发言 _idx 集合”做差集，看有哪些没被覆盖
        unclustered_idx = batch_original_idx - covered_idx

        tqdm.write(
            f"[批次 {b+1}] 话题簇覆盖原始发言条数：{len(covered_idx)}，"
            f"未被覆盖条数：{len(unclustered_idx)}"
        )

        # ⭐⭐ 新增：打印“未被话题簇覆盖”的发言信息（最多前 20 条）
        if PRINT_UNCLUSTERED and unclustered_idx:
            # 先把这一批中未覆盖的发言对象捞出来
            unclustered_records = []
            for line in batch_lines:
                try:
                    obj = json.loads(line)
                except Exception:
                    continue
                idx_val = obj.get("_idx")
                try:
                    idx_int = int(idx_val)
                except Exception:
                    continue
                if idx_int in unclustered_idx:
                    unclustered_records.append(obj)

            # 按 _idx 排序，避免乱序
            unclustered_records.sort(key=lambda x: int(x.get("_idx", 0)))

            print(f"\n[批次 {b+1}] 未被话题簇覆盖的原始发言（最多前20条）：")
            for rec in unclustered_records[:20]:
                idx = rec.get("_idx")
                date = rec.get("发言日期") or rec.get("日期") or ""
                t = rec.get("发言时间") or rec.get("时间") or ""
                speaker = rec.get("发言人ID") or rec.get("玩家ID") or rec.get("角色ID") or ""
                msg = (
                    rec.get("玩家消息")
                    or rec.get("发言内容")
                    or rec.get("玩家发言")
                    or rec.get("消息")
                    or ""
                )
                print(f"- _idx={idx} [{date} {t}] {speaker}: {msg}")
            print()  # 换行分隔一下

    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        continue

    # 防止 QPS 过高
    time.sleep(SLEEP_BETWEEN)

print("\n✅ 全部批次处理完成！")
print(f"原始输入总数：{total}")
print(f"模型#1 筛相关后总行数：{total_filtered_lines}，其中玩家发言：{total_filtered_player_lines} 条")
print(f"模型#2 话题簇累计覆盖原始发言条数：{total_covered_idx}")

准备处理 141 条，共 1 批（每批 300 条）。


🔥 批处理进度:   0%|                                                                                                                                     | 0/1 [02:39<?, ?批/s]      

[批次 1] 模型#1 输出总行数：89，其中玩家发言：89 条（不去重）


🔥 批处理进度:   0%|                                                                                                                                     | 0/1 [02:46<?, ?批/s]      


===== 批次 1/1 话题簇输出 =====
{"话题簇":"停服维护补偿讨论（2025-11-19 15:02:03-15:11:27）","核心对象/机制":"停服维护补偿机制，玩家讨论维护时长与补偿期望（100抽、50抽、石币、代币等）"}
{"话题簇":"游戏下载与安装问题（2025-11-19 15:02:22-15:13:20）","核心对象/机制":"游戏安装包下载与更新机制，玩家询问下载链接、是否需要重新下载等问题"}
{"话题簇":"角色技能更新讨论（2025-11-19 15:02:58-15:04:02）","核心对象/机制":"猛犸象和仙人掌角色的PTSD与强迫症被动技能更新实装情况"}
{"话题簇":"客服互动与调侃（2025-11-19 15:07:27-15:14:50）","核心对象/机制":"客服服务体验，玩家与客服薏米、蓝桉的互动调侃及无关游戏内容的闲聊"}

[批次 1] 话题簇覆盖原始发言条数：0，未被覆盖条数：141


🔥 批处理进度: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [02:47<00:00, 167.86s/批]


✅ 全部批次处理完成！
原始输入总数：141
模型#1 筛相关后总行数：89，其中玩家发言：89 条
模型#2 话题簇累计覆盖原始发言条数：0


## 加讨论观点分析的版本测试

In [ ]:
from tqdm import tqdm
import time
import json
from pathlib import Path

# --- 前置：系统提示、初始化等 ---
system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 做话提簇
system_prompt03 = load_system_prompt(PROMPT_MD_PATH03)  # 日话题簇聚合
system_prompt04 = load_system_prompt(PROMPT_MD_PATH04)  # 版本聚合（模型4）

batch_cluster_outputs = []  # 用于存放当天所有批次的话题簇 JSON 行（给模型#3用）

total = len(jsonl_lines01)
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

written_total = 0

# ====================================================
# 模型 #1 + 模型 #2：筛相关 + 话题簇划分（按批处理）
# ====================================================
for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    try:
        # --- 模型 #1：筛相关 ---
        user_prompt1 = build_user_prompt_filter(batch_lines)
        output_filter = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt01,
            user_prompt=user_prompt1,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        
        if not output_filter:
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue

        filter_count = sum(1 for line in output_filter.splitlines() if line.strip())
        tqdm.write(f"[批次 {b+1}] 模型#1 筛后保留 {filter_count} 条")
        written_total += filter_count

        # --- 模型 #2：话题簇划分 ---
        user_prompt2 = build_user_prompt_clsuter(output_filter)
        output_cluster = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt02,
            user_prompt=user_prompt2,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )

        if not output_cluster:
            tqdm.write(f"[批次 {b+1}] 模型#2 无有效输出，跳过。")
            continue

        # ---------- ✅ 新的 JSON 解析 + _cluster_id 部分 ----------
        try:
            # 1) 先用修复解析器，把模型#2输出变成 list[dict]
            cluster_json_list = parse_model2_output_to_json_list(
                output_cluster,
                batch_idx=b+1,
            )

            if not cluster_json_list:
                tqdm.write(f"[批次 {b+1}] ⚠ 本批次未解析出任何合法 JSON 对象，跳过。")
                continue

            # 2) 从该批次中推断日期（你自己的逻辑）
            date_str = infer_date_for_batch(cluster_json_list, batch_lines)
            batch_id = f"B{b+1}"

            # 3) 分配全局 _cluster_id：日期 + 批次号 + 序号 ...
            cluster_json_list = assign_global_cluster_ids(
                cluster_json_list,
                date_str,
                batch_id,
            )

            # 4) 再转成 jsonl 文本（供后续 aggregate_cluster_outputs 使用）
            output_cluster_with_ids = "\n".join(
                json.dumps(c, ensure_ascii=False) for c in cluster_json_list
            )
            batch_cluster_outputs.append(output_cluster_with_ids)

        except Exception as e:
            tqdm.write(f"[批次 {b+1}] ⚠ 添加 _cluster_id 失败：{e}")
            continue
        # ---------- JSON 解析部分结束 ----------

    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        continue

    time.sleep(SLEEP_BETWEEN)
            

# ====================================================
# 模型 #3：日话题簇聚合
# ====================================================

# --- 聚合当天所有批次的话题簇 jsonl ---
all_cluster = aggregate_cluster_outputs(batch_cluster_outputs)

# --- 模型 #3：日话题簇聚合 ---
user_prompt3 = build_user_prompt_cluster_agg(all_cluster)
output_cluster_agg = call_ark_chat_completions(
    api_url=API_URL,
    api_key=API_KEY,
    model=V3_1_MODEL_ID,           # 智能体3用的模型
    system_prompt=system_prompt03,  # 日话题簇聚合提示词
    user_prompt=user_prompt3,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    timeout=TIMEOUT_SEC,
    retries=RETRIES,
)

# 子话题簇（模型#2 输出，经 aggregate 后的 all_cluster，本身就是 jsonl）
parsed_subclusters = [
    json.loads(line.strip())
    for line in all_cluster.strip().splitlines()
    if line.strip()
]

# 聚合簇（模型#3 输出，如果不太干净，可以用 parse_jsonl_text 包一下）
parsed_clusters, parsed_subclusters_enriched = normalize_model3_clusters(output_cluster_agg, parsed_subclusters)






# ====================================================
# 计算 Top5 & 写入版本 jsonl
# ====================================================
top5_results = extract_top5_heat_clusters(parsed_clusters, jsonl_lines01, top_k=5)
final_result_all = attach_discussion_points(top5_results, parsed_subclusters_enriched)




# 输出检查（可暂时打开）
# for row in final_result:
#     print(json.dumps(row, ensure_ascii=False, indent=2))

append_daily_top5_to_version_jsonl(
    final_result_all,
    version_jsonl_path=VERSION_TOP5_JSONL,  # 这个路径你可以按版本号动态改
)



准备处理 47104 条，共 158 批（每批 300 条）。


🔥 批处理进度:   0%|                                                                                                                                                                              | 0/158 [00:00<?, ?批/s]

In [5]:
from tqdm import tqdm
import time
import json
from pathlib import Path

# =========================
# 1. 基础路径配置
# =========================
system_prompt04 = load_system_prompt(PROMPT_MD_PATH04)  # 版本聚合（模型4）
system_prompt05 = load_system_prompt(PROMPT_MD_PATH05)  # 讨论点聚合
system_prompt06 = load_system_prompt(PROMPT_MD_PATH06)  #讨论点分析
system_prompt07 = load_system_prompt(PROMPT_MD_PATH07)  #热度趋势分析

# =========================
# 2. 读取多日 daily_TOP5
# =========================
DAILY_TOP5_PATH = Path(VERSION_TOP5_JSONL)          # 你现有的多日 top5 文件路径，比如 "daily_TOP5.jsonl"
VERSION_AGG_INPUT_PATH = BASE_DIR / "version_agg_input.jsonl"  # 抽取后的输入文件（给模型#4，方便排查）
  # 抽取后的输入文件（给模型#4）

# 1) 读取多日 daily_TOP5
if not DAILY_TOP5_PATH.exists():
    raise FileNotFoundError(f"找不到 daily_TOP5 文件：{DAILY_TOP5_PATH}")

daily_top5 = read_jsonl_file(DAILY_TOP5_PATH)
if not daily_top5:
    raise ValueError("daily_TOP5 文件为空或无可解析 JSON 行，无法做版本聚合。")

# =========================
# 3. 构建版本聚合输入（给模型4）
# =========================
agg_top5_text = build_version_agg_input_jsonl_text(daily_top5, max_points_per_row=3)

if not agg_top5_text.strip():
    raise ValueError("抽取后 agg_top5_text 为空：请检查 daily_TOP5 每行是否都有 _daily_top_id/日期/聚合话题簇/讨论点。")

VERSION_AGG_INPUT_PATH.parent.mkdir(parents=True, exist_ok=True)
VERSION_AGG_INPUT_PATH.write_text(agg_top5_text + "\n", encoding="utf-8")
version_agg_input_text = agg_top5_text


# =========================
# 4. 模型4：版本级聚合话题簇
# =========================
user_prompt4 = build_user_prompt_version_agg(version_agg_input_text)
output_version_agg = call_ark_chat_completions(
    api_url=API_URL,
    api_key=API_KEY,
    model=V3_1_MODEL_ID,
    system_prompt=system_prompt04,
    user_prompt=user_prompt4,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    timeout=TIMEOUT_SEC,
    retries=RETRIES,
)

version_clusters: list[dict] = []   # ✅ 用内置类型，运行时不会 NameError
for i, line in enumerate(output_version_agg.strip().splitlines(), start=1):
    s = line.strip()
    if not s:
        continue
    try:
        obj = json.loads(s)
    except Exception as e:
        print(f"[模型#4版本聚合输出] 第{i}行 JSON解析失败：{e}\n原文：{s[:200]}...")
        continue
    version_clusters.append(obj)

# ✅ 直接用前面读过的 daily_top5
#version_top5 = compute_version_heat_topk(
    #version_clusters,
    #daily_top5,   # 这里用的是内存里的 daily_top5
    #top_k=5,
#)

# =========================
# 5. 模型5：对每个讨论点做聚合
# =========================
clusters_jsonl_text = clusters_list_to_jsonl(version_clusters)
model5_outputs: list[dict] = []

for i, cluster in enumerate(tqdm(version_clusters, desc="模型#5 聚合中"), start=1):
    try:
        user_prompt5 = json.dumps(cluster, ensure_ascii=False)

        response_text = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt05,
            user_prompt=user_prompt5,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )

        obj = json.loads(response_text.strip())
        model5_outputs.append(obj)

    except json.JSONDecodeError as e:
        print(f"[模型#5 输出JSON解析失败] 第{i}个聚合簇：{e}\n原文：{response_text[:200]}...")
        continue
    except Exception as e:
        print(f"[模型#5 调用失败] 第{i}个聚合簇：{e}")
        continue

# =========================
# # ⭐⭐⭐ 后续一切版本级 Top5 计算，都基于“模型#5 聚合后的版本簇”
# =========================
if model5_outputs:
    fused_version_clusters = model5_outputs
else:
    # 兜底：如果模型5全部失败，就退回使用模型4的原始 version_clusters
    print("⚠ 模型#5 没有成功输出，退回模型#4的版本话题簇。")
    fused_version_clusters = version_clusters


# ====================================================
# ⭐⭐⭐⭐⭐⭐版本级话题簇发言热度 Top5
# ====================================================
# 1) 将idx与时间轴对齐→后续回溯原文
cluster_point_stats = compute_all_clusters_point_metrics(
    version_clusters=fused_version_clusters,
    daily_top5_rows=daily_top5,
    raw_jsonl_lines=jsonl_lines01,
)
# 2) 按讨论点量化结果，算出“版本话题簇Top5”
version_top5_points = extract_version_top5_clusters_from_point_metrics(
    clusters_with_points=cluster_point_stats,
    top_k=5,)


# =========================
# 7. 版本top5发言话题簇量化数据提取
# =========================
heated_stats = build_cluster_heat_summary(version_top5_points)


# ====================================================
# 模型六: top5热度话题簇下讨论点&发言示例分析
# ====================================================
#1) 版本top5讨论点拉平
rows_version = print_mech_time_from_version_top5(version_top5_points)
version_opinions = []
for r in rows_version:
    mech = r["讨论点"]
    dt_list = r["日期时间轴列表"]  # 多段日期+时间轴

    dialogs_lines = get_dialogs_lines_by_dt_list_debug(
        raw_jsonl_lines=jsonl_lines01,
        dt_list=dt_list,
        debug=True,
    )

    if not dialogs_lines:
        print(f"⚠ 该讨论点未匹配到原文，跳过：{mech}")
        continue
    #2）模型6:版本玩家观点&发言示例分析
    user_prompt6 = build_user_prompt_version_opinion(discussion_point=mech,
        json_lines=dialogs_lines,)
    try:
        output_version_opinion = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt06,
            user_prompt=user_prompt6,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
    except Exception as e:
        print("❌ 调用模型4出错:", e)
        continue

    opinions_version_mech = parse_opinion_output_to_list(output_version_opinion)
    for op in opinions_version_mech:
        op["讨论点"] = mech
    
    version_opinions.extend(opinions_version_mech)

# 3)版本讨论点与玩家观点回溯至输出
merged_opinion_version = merge_version_top5_with_opinions(
version_top5_points=version_top5_points,
version_opinions=version_opinions,
)

# =========================
# 模型7：取话提簇和讨论点信息用于热度趋势分析（作为模型7输入）
# =========================



# 1）先构建 JSONL 文本（每行一个聚合话题簇对象）
stats_jsonl_text = build_heat_trend_input_jsonl(version_top5_points)
heat_trend_results: list[dict] = []

# 2）按行拆开，逐条送入模型#7
lines = [ln.strip() for ln in stats_jsonl_text.splitlines() if ln.strip()]
for i, line in enumerate(lines, start=1):
    # line 形如：{"聚合话题簇": "...", "讨论点列表": [...]}

    # （可选）先确保这一行本身是合法 JSON，方便排错
    try:
        _ = json.loads(line)
    except Exception as e:
        print(f"[模型#7 输入校验失败] 第{i}行不是合法 JSON：{e}\n原文：{line[:200]}...")
        continue

    # 构造用户提示词：只带这一行
    user_prompt7 = build_user_prompt_heat_trend(line)

    # 调模型#7
    try:
        resp_text = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt07,   # 你前面 load_system_prompt 出来的热度趋势智能体 prompt
            user_prompt=user_prompt7,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
    except Exception as e:
        print(f"[模型#7 调用失败] 第{i}行：{e}")
        continue
    
    obj = json.loads(resp_text.strip())
    topic = (obj.get("聚合话题簇") or "").strip()
    summary = (obj.get("一句话总结") or "").strip()
    if not topic or not summary:
        print(f"[模型#7 输出缺字段] 第{i}行：{obj}")
        continue

    heat_trend_results.append({
        "聚合话题簇": topic,
        "一句话总结": summary,
    })

final_version_outputs = merge_version_final_summary(
    merged_opinion_version=merged_opinion_version,
    heated_stats=heated_stats,
    heat_trend_results=heat_trend_results,
)

模型#5 聚合中: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [01:03<00:00,  3.75s/it]


🔎 时间片 2025-12-03 10:04:05-10:31:25 命中 172 条消息
🔎 时间片 2025-12-03 10:34:35-10:57:21 命中 268 条消息
✅ 共合并出 440 条消息（已去重）
🔎 时间片 2025-12-03 10:49:56-10:50:13 命中 10 条消息
✅ 共合并出 10 条消息（已去重）
🔎 时间片 2025-12-03 14:11:59-14:22:52 命中 111 条消息
🔎 时间片 2025-12-03 14:52:50-15:01:51 命中 116 条消息
🔎 时间片 2025-12-03 17:20:06-17:22:56 命中 13 条消息
✅ 共合并出 240 条消息（已去重）
🔎 时间片 2025-12-03 14:10:30-14:33:47 命中 223 条消息
🔎 时间片 2025-12-03 16:38:39-16:39:23 命中 11 条消息
🔎 时间片 2025-12-03 16:49:08-16:49:47 命中 10 条消息
✅ 共合并出 244 条消息（已去重）
🔎 时间片 2025-12-04 08:01:26-09:14:23 命中 89 条消息
🔎 时间片 2025-12-04 10:16:10-10:16:32 命中 3 条消息
🔎 时间片 2025-12-06 11:53:59-12:52:58 命中 107 条消息
🔎 时间片 2025-12-07 21:46:29-22:05:55 命中 32 条消息
✅ 共合并出 231 条消息（已去重）
🔎 时间片 2025-12-04 10:17:29-10:21:51 命中 20 条消息
✅ 共合并出 20 条消息（已去重）
🔎 时间片 2025-12-02 16:28:26-16:28:49 命中 3 条消息
✅ 共合并出 3 条消息（已去重）
🔎 时间片 2025-12-02 16:30:01-17:42:41 命中 20 条消息
✅ 共合并出 20 条消息（已去重）
🔎 时间片 2025-12-03 14:20:16-14:34:45 命中 145 条消息
✅ 共合并出 145 条消息（已去重）
🔎 时间片 2025-12-03 14:39:57-14:44:49 命中 27 条消息
✅ 共合并出 27 

In [6]:
final_version_outputs

[{'话提簇标题': '游戏下载与安装',
  '讨论热度（量化）': {'讨论覆盖天数': '1天（2025-12-03 ~ 2025-12-03）',
   '发言玩家总数': 47,
   '发言总量': 450,
   '热度评分': 932.44,
   '热度趋势': '该话题在12月3日集中爆发，玩家主要围绕测试版本下载链接获取时间与安装方式密集提问，热度呈现单日高峰后快速消退，反映出对首波测试参与节奏的高度关注。'},
  '讨论点列表': [{'讨论点1': '测试版本下载链接获取时间与安装方式',
    '玩家观点': ['1：多名玩家反复询问下载方式和安装包获取时间，表现出对测试版本下载信息的急切需求',
     '2：若干玩家确认下载时间为下午，并询问具体时间点（如1:50或2点）',
     '3：个别玩家询问是否需要额外资格或注册账号，对测试资格有疑问',
     '4：少数玩家询问安装包下载后是否可以直接进入游戏，对安装流程有疑问'],
    '代表性玩家发言示例': ['怎么下载呀',
     '软件那里下？',
     '发安装包了会@全体吗',
     '下载了安装包就可以直接进去了吗',
     '不需要资格什么的吗']},
   {'讨论点2': '安装包发放时的全体通知机制与免打扰设置',
    '玩家观点': ['1：玩家询问安装包发放时是否会使用@全体通知功能，以决定是否开启免打扰设置',
     '2：玩家表示如果会@全体通知，就需要开启免打扰功能'],
    '代表性玩家发言示例': ['发安装包了会@全体吗', '会的话我要开免打扰了', 'okok']}]},
 {'话提簇标题': '猜硬币概率机制',
  '讨论热度（量化）': {'讨论覆盖天数': '1天（2025-12-03 ~ 2025-12-03）',
   '发言玩家总数': 33,
   '发言总量': 240,
   '热度评分': 511.23,
   '热度趋势': '该话题在12月3日集中爆发，玩家主要围绕猜硬币游戏的随机概率与奖励机制密集讨论，呈现短期高度关注后快速消退的特点，反映出对游戏核心玩法公平性与收益透明度的强烈关注。'},
  '讨论点列表': [{'讨论点1': '猜硬币游戏的随机概率机制与奖励机制',

In [7]:
daily_top5

[{'聚合话题簇': 'PVP玩法机制与平衡性讨论',
  '日期': '2025-12-02',
  '时间轴': '10:23:19-12:43:56、15:57:15-16:09:24、16:21:21-16:21:33',
  '发言玩家总数': 17,
  '发言总数': 90,
  '热度评分': 161.28,
  '讨论点': [{'讨论点': '新增PVP玩法对挂机玩家资源影响、战力平衡性、重启玩家优势等实战影响的讨论',
    '日期时间轴列表': [{'日期': '2025-12-02', '时间轴': '10:23:19-12:43:56'}],
    '子话题簇列表': ['2025-12-02_B1_07']},
   {'讨论点': 'PVP玩法中星级压制机制、开局装备搭配（血套vs棒球套）对战力平衡的影响分析',
    '日期时间轴列表': [{'日期': '2025-12-02', '时间轴': '15:57:15-16:09:24'}],
    '子话题簇列表': ['2025-12-02_B1_12']},
   {'讨论点': '建议PVP玩法中添加击杀大佬的记录功能用于社交炫耀',
    '日期时间轴列表': [{'日期': '2025-12-02', '时间轴': '16:21:21-16:21:33'}],
    '子话题簇列表': ['2025-12-02_B1_13']}],
  '_idx': 1,
  '_daily_top_id': '2025-12-02_T01'},
 {'聚合话题簇': '停服维护公告与账号权限咨询',
  '日期': '2025-12-02',
  '时间轴': '09:59:18-10:08:51、10:04:53-10:08:37、12:36:02-12:38:16',
  '发言玩家总数': 15,
  '发言总数': 33,
  '热度评分': 86.17,
  '讨论点': [{'讨论点': '12月3日10:00-14:00停服维护及开服时间公告，包含新功能PVP玩法预告',
    '日期时间轴列表': [{'日期': '2025-12-02', '时间轴': '09:59:18-10:08:51'}],
    '子话题簇列表': ['2025-12-02_B

In [14]:
merged_opinion_version

[{'聚合话题簇': '游戏下载与测试资格获取',
  '日期': '2025-12-03',
  '时间轴': '10:04:05-10:31:25、10:34:35-10:57:21、10:49:56-10:50:13、10:14:51-10:16:19、10:51:21-10:52:19、10:58:38-11:01:33',
  '讨论点列表': [{'讨论点1': '测试版本下载链接发放时间与安装包获取方式',
    '玩家观点': ['1：多名玩家询问下载方式和安装包获取时间，表现出对测试版本下载流程的急切关注',
     '2：若干玩家询问是否需要测试资格，对参与测试的条件存在疑问',
     '3：个别玩家询问安装包发放后是否会@全体成员，关注通知方式'],
    '代表性玩家发言示例': ['怎么下载呀', '软件那里下？', '发安装包了会@全体吗']},
   {'讨论点2': '安装包发放时的全体通知机制与免打扰设置',
    '玩家观点': ['1：玩家关心安装包发放时是否会使用@全体通知功能，以便提前做好免打扰设置',
     '2：玩家表示如果会@全体，就需要开启免打扰功能'],
    '代表性玩家发言示例': ['发安装包了会@全体吗', '会的话我要开免打扰了', 'okok']},
   {'讨论点3': '测试资格获取条件、账号注册要求及奖励处理规则',
    '玩家观点': ['1：多名玩家询问测试资格获取方式，关注是否还有资格以及如何获取',
     '2：若干玩家对账号注册要求存在疑问，担心停服后是否需要重新注册账号',
     '3：个别玩家关注奖励处理规则，担心强制重启会导致奖励丢失或需要重新获取',
     '4：个别玩家确认进入群聊即代表拥有测试资格，表示理解资格获取规则'],
    '代表性玩家发言示例': ['还有资格吗', '啥意思停服要重新注册账号吗', '强制重启不会补发重启奖励吧', '进了群就代表有资格了']}]},
 {'聚合话题簇': '棒棒糖货币使用与商店策略',
  '日期': '2025-12-04、2025-12-03',
  '时间轴': '11:21:14-11:22:19、11:47:14-11:48:23、11:48:16-11:49:34、14:10:30

## 开始运行(不可动）

In [ ]:
from tqdm import tqdm
import time
import json
from pathlib import Path

# ... 前置：系统提示、create_intent_excel_styled(EXCEL_FILE)、jsonl_lines01 等

system_prompt01 = load_system_prompt(PROMPT_MD_PATH01)  # 筛相关
system_prompt02 = load_system_prompt(PROMPT_MD_PATH02)  # 做话提簇
system_prompt03 = load_system_prompt(PROMPT_MD_PATH03)  # 话题簇聚合 / 校正（智能体4）
system_prompt04 = load_system_prompt(PROMPT_MD_PATH04)  # 话提簇玩家观点感受分析


sub_opinion_map = {}
# 用于存放当天所有批次的话题簇 JSON 行（给智能体4用）
batch_cluster_outputs  = []
total = len(jsonl_lines01)
if total == 0:
    print("没有可处理的数据。")
else:
    total_batches = (total + BATCH_SIZE - 1) // BATCH_SIZE
    print(f"准备处理 {total} 条，共 {total_batches} 批（每批 {BATCH_SIZE} 条）。")

written_total = 0

for b in tqdm(range(total_batches), desc="🔥 批处理进度", unit="批"):
    start = b * BATCH_SIZE
    end = min(start + BATCH_SIZE, total)
    batch_lines = jsonl_lines01[start:end]

    try:
        # --- 模型 #1：筛相关 ---
        user_prompt1 = build_user_prompt_filter(batch_lines)
        output_filter = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_MODEL_ID,
            system_prompt=system_prompt01,
            user_prompt=user_prompt1,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        
        if not output_filter:
            tqdm.write(f"[批次 {b+1}] 模型#1 无有效输出，跳过。")
            continue
        filter_count = sum(1 for line in output_filter.splitlines() if line.strip())
        tqdm.write(f"[批次 {b+1}] 模型#1 筛后保留 {filter_count} 条")
        written_total += filter_count   # 如果不需要总数，可以删掉这一行
        
        # --- 模型 #2：话题簇划分 ---
        user_prompt2 = build_user_prompt_clsuter(output_filter)
        output_cluster = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,
            system_prompt=system_prompt02,
            user_prompt=user_prompt2,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )
        
        

        if not output_cluster:
            tqdm.write(f"[批次 {b+1}] 模型#2 无有效输出，跳过。")
            continue
        try:
            cluster_json_list = []
            for line in output_cluster.strip().splitlines():
                line = line.strip()
                if not line.startswith("{"):
                    continue
                obj = json.loads(line)
                # 将 "话题簇1" → "话题簇"，便于后续统一处理
                for key in list(obj.keys()):
                    if key.startswith("话题簇") and key != "话题簇":
                        obj["话题簇"] = obj.pop(key)
                cluster_json_list.append(obj)

            # 假设从输出中拿日期
            date_str = infer_date_for_batch(cluster_json_list, batch_lines)  # 你可以从上下文获得更准确值
            batch_id = f"B{b+1}"
            cluster_json_list = assign_global_cluster_ids(cluster_json_list, date_str, batch_id)
        
            output_cluster_with_ids = "\n".join(json.dumps(c, ensure_ascii=False) for c in cluster_json_list)
            batch_cluster_outputs.append(output_cluster_with_ids)
        except Exception as e:
            tqdm.write(f"[批次 {b+1}] ⚠ 添加 _cluster_id 失败：{e}")
            continue

       
       
    except Exception as e:
        tqdm.write(f"[批次 {b+1}] ❌ 出错：{e}")
        continue

    time.sleep(SLEEP_BETWEEN)



# --- 每日话提簇聚合成jsonl ---
all_cluster = aggregate_cluster_outputs(batch_cluster_outputs)

# --- 模型 #3：日话提簇聚合 ---
user_prompt3 = build_user_prompt_cluster_agg(all_cluster)
output_cluster_agg = call_ark_chat_completions(
    api_url=API_URL,
    api_key=API_KEY,
    model=V3_1_MODEL_ID,           # 或你给智能体4选的模型
    system_prompt=system_prompt03,  # 话题簇聚合/校正提示词
    user_prompt=user_prompt3,
    temperature=TEMPERATURE,
    max_tokens=MAX_TOKENS,
    timeout=TIMEOUT_SEC,
    retries=RETRIES,
)

#每日总发言转单条jonsl格式
parsed_subclusters = [
    json.loads(line.strip()) for line in all_cluster.strip().splitlines() if line.strip()
]

# 解析原始发言 jsonl，为按时间轴筛选对话用
parsed_msgs = [json.loads(line.strip()) for line in jsonl_lines01 if line.strip()]

#每日聚合后话提簇转单条jsonl格式
parsed_clusters = [json.loads(line) for line in output_cluster_agg.strip().splitlines() if line.strip()]

#计算每日讨论热度top5
top5_results = extract_top5_heat_clusters(parsed_clusters, jsonl_lines01, top_k=5)

#######提取子话提簇讨论点###########################################################
####### 建子簇索引：_cluster_id -> 子簇对象 ########
sub_map = {}
for row in parsed_subclusters:
    cid = row.get("_cluster_id")
    if isinstance(cid, str) and cid:
        sub_map[cid] = row

####### 在 Top5 循环里，对每个【子话题簇】单独跑模型#4 ########
for cluster in top5_results:
    date = cluster["日期"]
    # 聚合簇的整体时间轴（只用来兜底）
    cluster_time_axis = cluster["时间轴"]
    cid_list = cluster.get("子话题簇列表", [])

    for cid in cid_list:
        sub = sub_map.get(cid)
        if not sub:
            print(f"\n[聚合话题簇：{cluster.get('聚合话题簇')}] 子话题簇 ID：{cid}")
            print("  ⚠ 在 sub_map / parsed_subclusters 中未找到该 _cluster_id。")
            continue

        
        # ---- 1）从子话题簇【标题】里提取时间轴 ----
        sub_title = sub.get("话题簇", "") or ""
        sub_time_axis = extract_time_axis_from_title(sub_title)

        # ---- 2）按【子话提簇时间轴】从原始发言里匹配 dialogs ----
        dialogs = match_dialogs_by_time(parsed_msgs, date, sub_time_axis)

        # ---- 3）构造 discussion_point（不打印，只用来写 prompt）----
        discussion_point = (
    
            sub.get("核心对象/机制")
            or sub.get("话题簇")
            or ""
        )
        topic_id = sub.get("_cluster_id", cid)
        
        # ---- 4）模型4 玩家发言感受分析总结----
        user_prompt4 = build_user_prompt_subcluster_opinion(
            topic_id=topic_id,
            discussion_point=discussion_point,
            dialogs=dialogs,
        )
        if not isinstance(user_prompt4, str) or not user_prompt4.strip():
            print(f"  ⚠ user_prompt4 为空或非法，跳过本子话题簇：{cid}")
            continue
        opinion_output = call_ark_chat_completions(
            api_url=API_URL,
            api_key=API_KEY,
            model=V3_1_MODEL_ID,             
            system_prompt=system_prompt04,   
            user_prompt=user_prompt4,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            timeout=TIMEOUT_SEC,
            retries=RETRIES,
        )


       
    opinion_obj = parse_and_normalize_opinion_output(
        opinion_output=opinion_output,
        topic_id=topic_id,
        discussion_point=discussion_point,
    )
    sub_opinion_map[topic_id] = opinion_obj
# ===================================
# 组装 Top5 + 观点总结，并写入 jsonl
# ===================================



准备处理 2308 条，共 8 批（每批 300 条）。


🔥 批处理进度:   0%|                                                                                                                                                       | 0/8 [06:33<?, ?批/s]      

[批次 1] 模型#1 筛后保留 240 条


🔥 批处理进度:  12%|█████████████████▊                                                                                                                            | 1/8 [12:29<48:22, 414.62s/批]      

[批次 2] 模型#1 筛后保留 206 条


🔥 批处理进度:  25%|███████████████████████████████████▌                                                                                                          | 2/8 [20:44<38:45, 387.59s/批]      

[批次 3] 模型#1 筛后保留 283 条


🔥 批处理进度:  38%|█████████████████████████████████████████████████████▎                                                                                        | 3/8 [21:22<36:31, 438.36s/批]

In [9]:
final_top5_with_opinion = build_daily_top5_opinion_records(
    top5_results=top5_results,
    sub_opinion_map=sub_opinion_map,
)

append_daily_top5_to_version_jsonl(
    final_result=final_top5_with_opinion,
    version_jsonl_path="version_daily_top5_with_opinion.jsonl",
)

✅ 已将当日 Top5（含 _idx 和 _daily_top_id）追加写入: version_daily_top5_with_opinion.jsonl


In [23]:
print(cid_list)
print(sub)

print(sub_title)
print(sub_time_axis)
print(discussion_point)
print(dialogs)
print(cid)

['2025-12-02_B1_03', '2025-12-02_B1_14', '2025-12-02_B2_07']
{'话题簇': '游戏物品异常问题反馈（2025-12-02 22:57:27-22:57:31）', '核心对象/机制': '游戏内物品异常消失问题的反馈与处理', '_cluster_id': '2025-12-02_B2_07'}
游戏物品异常问题反馈（2025-12-02 22:57:27-22:57:31）
22:57:27-22:57:31
游戏内物品异常消失问题的反馈与处理
[{'发言日期': '2025-12-02', '发言时间': '22:57:27', '玩家ID': '无为无畏(2514177080)', '玩家消息': '[图片]'}, {'发言日期': '2025-12-02', '发言时间': '22:57:31', '玩家ID': '无为无畏(2514177080)', '玩家消息': '东西没了'}]
2025-12-02_B2_07


In [6]:
print(opinion_output)

NameError: name 'opinion_output' is not defined

In [14]:
print(sub_time_axis)